In [2]:
%load_ext autoreload
%autoreload 2

1. sharing internet on the laocal machine

``
    ifconfig
``

2. scanning the local network to find the raspberry π

``
    nmap -T4 -sP 192.168.2.0/24
``

3 . cloned the repository on the π

````
  686  pip install pyserial
  687  vim .ssh/id_rsa.pub
  688  vim .ssh/id_rsa
  689  chmod 0600 .ssh/id_rsa
  690  git clone git@git.framasoft.org:laurentperrinet/elasticte.git
  691  cd elasticte/
  692  ls
  693  cat scenario_line_contraint.py
  694  python scenario_line_contraint.py  serial
  695  pip install -e .
  696  pip install --user -e .
  697  python scenario_line_contraint.py  serial
``


In [4]:
duration = el.get_default_args(el.EdgeGrid.render)['duration']

In [5]:

class EdgeGrid(el.EdgeGrid):
    def update_particles(self):
        self.particles[2,:] =  .5 + .5*np.cos((self.t + np.linspace(0, 1., self.N_particles))*2*np.pi)

    def update(self):
        self.update_particles()

e = EdgeGrid(N_lame=25, grid_type='line')
fig, ax = plt.subplots(1, 1, figsize=(e.figsize, e.figsize))
#ax = fig.add_axes((0, 0, 1, 1), axisbg='w')
fps = 10.
e.dt = 1./fps
opts = dict(vmin=0, vmax=1., linewidths=0, cmap=plt.cm.bone)
#print(opts)

In [6]:
def make_frame_mpl(t):    
    ax.clear()
    ax.axis('off')
    e.t = t
    e.update()
    scat  = ax.scatter(e.particles[0,:], e.particles[1,:], c=e.particles[2,:], **opts)
    return mplfig_to_npimage(fig) # RGB image of the figure

animation = mpy.VideoClip(make_frame_mpl, duration=duration)
animation.ipython_display(fps=fps, loop=1, autoplay=1)

NameError: name 'mpy' is not defined

## git


In [63]:
!git s
#!git add 2015-10-27\ élasticité\ expansion.ipynb

 M "2015-10-27 \303\251lasticit\303\251 expansion.ipynb"
?? ../.ipynb_checkpoints/
?? ../PLAN_PRINCIPE_PROTO/
?? ../Presentation-Dossier-Artistique/
?? ../TESTS_TRAME/
?? ../__temp_ipython__.png
?? ../files/elastic_fresnel.mp4
?? ../files/elastic_fresnel_gauche.mp4
?? ../files/elastic_fresnel_lateral.mp4
?? ../files/elastic_fresnel_pasapas.mp4
?? ../files/elastic_fresnel_rot.mp4
?? ../files/fresnel.mp4
?? "../files/grille_carr\303\251e.mp4"
?? ../files/lames_contraintes.mp4
?? ../files/lames_oscillantes.mp4
?? ../files/lames_oscillantes_fps5.mp4
?? "../notes-Laurent/2015-10-22_\303\251lasticit\303\251 LAMES-Dynamique_pdf.pdf"
?? .ipynb_checkpoints/
?? __temp__.mp4
?? __temp_ipython__.png
?? anim.gif
?? ../screenshot.png
?? ../src/elasticite.egg-info/


In [15]:
!git commit -am' expansion - function de samples sur la structure'

[master 5217ce7]  expansion - function de samples sur la structure
 3 files changed, 1036 insertions(+), 1209 deletions(-)
 rewrite posts/2015-04-30_trame-sensorielle.ipynb (99%)


In [16]:
! git push


Counting objects: 17, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (17/17), done.
Writing objects: 100% (17/17), 76.41 KiB | 0 bytes/s, done.
Total 17 (delta 11), reused 0 (delta 0)
To git@git.framasoft.org:laurentperrinet/elasticte.git
   3f02e9c..5217ce7  master -> master
